### 1. Loading Data

The first step of training a machine learning algorithm is loading the training data. 
- Preload data into memory
    - The simplest method is to preload all your data into memory and pass it to TensorFlow as a single array
    - Simply __read your data file into an array__ to TensorFlow (size up to computer's available memory)
    - As long as the data ends up in a multidimensional array then you're good
        - Using pandas and preprocessing the data
    
    
- Feed data step by step
    - Write code that feeds your training data step-by-step into TensorFlow as TensorFlow requests it 
    - TensorFlow calls the data loader function whenever it needs the next chunk of data which gives you more control
    - Easier to process large datasets since it loads one chunk at a time
    - Have to write all of the code yourself
    
    
- Set up a custom data pipeline
    - This is the best option when you are working with enormous datasets like millions of images. A data pipeline allows TensorFlow to manage loading data into memory itself as it needs it
    - Data pipeline only loads data into memory in small chunks which means that it can work with large datasets
    - Requires writing TensorFlow-specific code
    - Big advantage of building a data pipeline is that you can take advantage of __parallel processing__ across multiple CPUs
    - Can have several threads running at the same time to load and preprocess data
    - Training process doesn't have to stop and wait while the next chunk of data is loaded for the next training pass

## 2. Define model parameters and create the layers

Our neural network should accept nine floating point numbers as the input for making predictions. But each time we want a new prediction the specific values we pass in will be different. So we can use a placeholder node to represent that. 

In [ ]:
# Define model parameters
learning_rate = 0.001
learning_epochs = 100
display_step = 5

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Define input layer (new variable scope)
# "None" tells TensorFlow our neural network can mix up batches of any size and number_of_inputs tells it to 
#  expect nine values for each record in the batch
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))
    
# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable('weights1', shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases1', shape=[layer_1_nodes], initializer=tf.zeros_initializer)
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases) # matrix multiplication and a standard rectified linear unit 

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable('weights2', shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases2', shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)
    
# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable('weights3', shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases3', shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
    
# Output layer   
with tf.variable_scope('output'):
    weights = tf.get_variable('weights4', shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases4', shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases
    
# Define the cost function
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))
    
# Define the optimizer function (train & optimize)
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

'''
    Training Session
    
    For session.run, the first command we always run is the built in command to tell TensorFlow to initialize 
    all variables in our graph to their default values. The command that's called tf.global_variables_initializer. 
    Now that all the variables in our graph are initialized, we're ready to create our training loop. 
'''

# Initialize a training session after defining the model
with tf.Session() as session:
    
    # Run the global variable initializer to initialize all variables/layers in the nn
    session.run(tf.global_variables_initializer())
    
    # Run the optimizer
    for epoch in range(2):
        
        # Add training data
        session.run('optimizer', feed_dict={X: X_scaled_training, Y: Y_scaled_training}) # Pass in operation 
        
        # Print progress
        if epoch % 5 == 0:
            training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
            testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
            print(epoch, training_cost, testing_cost)
    
    
    
    
    
    
    
    
    
    